# Dennis Guchu
# SCT213-C002-0006/2022
# Data Mining CAT 1

All tasks use the tips dataset (total bill, tip, sex, smoker, day, time, size). Each question must use results from the previous question.

(a) Standardize numeric columns (total bill, tip, size) and encode categorical (sex, smoker, day, time). Fit regression total bill = β0 + β1 tip + β2 size + β3 day/time. Diagnose multicollinearity and heteroscedasticity.

In [ ]:
#import required dependencies
import seaborn as sns
import pandas as pd
import numpy as np
import warnings

# Ignore all deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from sklearn.preprocessing import StandardScaler

In [ ]:
#load tips dataset
tips = sns.load_dataset("tips")
tips.head()

In [ ]:
#evaluate numerical vs categorical variables
tips.info()

In [ ]:
#standardize numeric variables
scaler = StandardScaler()

tips_scaled = tips.copy()
tips_scaled[['total_bill', 'tip', 'size']] = scaler.fit_transform(
    tips[['total_bill', 'tip', 'size']]
)

#encode categorical variables
tips_encoded = pd.get_dummies(
    tips_scaled,
    columns=['sex', 'smoker', 'day', 'time'],
    drop_first=True,
    dtype=int
)

tips_encoded.head()

In [ ]:
#regression model
X = tips_encoded.drop(columns='total_bill')
y = tips_encoded['total_bill']

X = sm.add_constant(X)


model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
#multicollinearity diagnosis
import matplotlib.pyplot as plt

corr = X.drop(columns='const').corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix of Predictors")
plt.show()

The correlation matrix of the predictors shows that no pair of explanatory variables has a very high correlation (|r| ≥ 0.8), which is the commonly used threshold for severe multicollinearity.

Overall, the correlation matrix indicates that multicollinearity is not a serious concern in this regression model, and the estimated coefficients can be interpreted reliably.

In [ ]:
#Heteroscedasticity diagnosis
fitted_vals = model.fittedvalues
residuals = model.resid

plt.figure(figsize=(7, 5))
plt.scatter(fitted_vals, residuals)
plt.axhline(0)
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted Values")
plt.show()

The spread of the residuals increases as the fitted values increase. Specifically, residuals are more tightly clustered around zero at lower fitted values, while a wider dispersion is observed at higher fitted values.

This fan-shaped pattern indicates that the variance of the error terms is not constant across all levels of the fitted values, suggesting the presence of heteroscedasticity in the regression model.

(b) Using residuals from (a), solve LP: Minimize Z = 4x1 +3x2 ; constraints 3x1 +2x2 ≥ 15, x1 + 4x2 ≥ 16, x1 , x2 ≥ 0. Interpret stochastic fractional allocation.

From (a), we fitted:

total_bill=β^0+β^1tip+β^2size+β^3(day/time)

Residuals:

ei=yi−y^i  



Objective

min Z = 4x_1 + 3x_2  

Constraints:

3x_1+2x_2≥15

X_1+4x_2≥16

X_1,x_2≥0





Solve the equalities:

3x_1+2x_2=15

X_1+4x_2=16



Multiply second equation by 3:

3x_1+12x_2=48

Subtract first equation:

(3x_1+12x_2)−(3x_1+2x_2)=48−15

10x_2 =33⇒x_2 =3.3  

Substitute back:

X_1+4(3.3)=16⇒x_1=2.8

Optimal Solution

X_1=2.8,  x_2=3.3

Minimum Cost:

Z=4(2.8)+3(3.3)=11.2+9.9=21.1



Interpretation

x_1 and x_2 are not integers

This implies divisible resources, not discrete units



Residuals from (a) indicate uncertainty in predicting total bill

LP variables represent probabilistic or fractional allocation of corrective effort

(c) Based on LP-adjusted predictions from (b), predict total bill and compute confidence/lift for sex=Male ⇒ time=Dinner. Assess robustness under small-sample bias.

From (b):

LP solution:

x1 =2.8,x2 =3.3  


LP-Adjusted Prediction

We adjust the regression prediction using a weighted residual correction:

y^ i(LP) =y^ i +λ1 x1 ei +λ2 x2 ei   

Since residuals represent unexplained variability, a simplified proportional adjustment is acceptable:

y^ i(LP) =y^ i +αei   

where

α=x1 +x2 / x1 +x2 + 1  = 6.1/ 7.1 ≈0.86

In [ ]:
#Predict Total Bill (Male, Dinner)
tips_lp = tips_encoded.copy()
tips_lp['y_hat'] = model.fittedvalues
tips_lp['residual'] = model.resid

alpha = (2.8 + 3.3) / (2.8 + 3.3 + 1)
tips_lp['y_hat_lp'] = tips_lp['y_hat'] + alpha * tips_lp['residual']

#filter male => Dinner
male_dinner = tips_lp[
    (tips['sex'] == 'Male') &
    (tips['time'] == 'Dinner')
]

predicted_total_bill = male_dinner['y_hat_lp'].mean()

#unscale total bill
tb_mean = scaler.mean_[0]   # mean of total_bill
tb_std  = scaler.scale_[0]  # std of total_bill

predicted_total_bill_unscaled = (predicted_total_bill * tb_std + tb_mean)

print(f"$ {predicted_total_bill_unscaled:.2f}")

Association Rule:

sex = Male ⇒ time = Dinner

We now compute confidence and lift.



Definitions

Let:

A: sex = Male

B: time = Dinner

Confidence

Conf(A⇒B)=P(B∣A)  

Lift

Lift(A⇒B)=P(B∣A)  / P(B)

In [ ]:
N = len(tips)

male = tips[tips['sex'] == 'Male']
dinner = tips[tips['time'] == 'Dinner']
male_dinner = tips[(tips['sex'] == 'Male') & (tips['time'] == 'Dinner')]

confidence = len(male_dinner) / len(male)
lift = confidence / (len(dinner) / N)

confidence, lift

In [ ]:
def compute_conf_lift(data):
  male = data[data['sex'] == 'Male']
  dinner = data[data['time'] == 'Dinner']
  male_dinner = data[(data['sex'] == 'Male') & (data['time'] == 'Dinner')]

  conf = len(male_dinner) / len(male)
  lift = conf / (len(dinner) / len(data))
  return conf, lift

conf_lift_samples = []

for _ in range(50):
  sample = tips.sample(frac=0.8, replace=False)
  conf_lift_samples.append(compute_conf_lift(sample))

conf_lift_samples = np.array(conf_lift_samples)

conf_std = conf_lift_samples[:,0].std()
lift_std = conf_lift_samples[:,1].std()

conf_std, lift_std

Low standard deviations of confidence and lift across subsamples confirm robustness to small-sample bias.

(d) Construct OLAP cube using predicted total bill from (c): day=Region, time=Month. Extract Sun-Dinner and discuss aggregation bias, sparsity, and dimensionality effects.

In [ ]:
# Continue from (c)
tips_olap = tips_lp.copy()

# Attach conceptual OLAP dimensions
tips_olap['Region'] = tips['day']
tips_olap['Month'] = tips['time']

# OLAP cube via pivot table
olap_cube = pd.pivot_table(
    tips_olap,
    values='y_hat_lp',
    index='Region',      # day → Region
    columns='Month',     # time → Month
    aggfunc='mean'
)

olap_cube

In [ ]:
#extract Sun-Dinner
sun_dinner = olap_cube.loc['Sun', 'Dinner']

# Recover scaler parameters
mean_bill = scaler.mean_[0]
std_bill = scaler.scale_[0]

sun_dinner_unscaled = mean_bill + sun_dinner * std_bill

print(f"$ {sun_dinner_unscaled:.2f}")

1. Aggregation bias is introduced when individual-level variation is collapsed into group means

High-spending Sunday dinner outliers are averaged with moderate spenders

LP-adjusted residual corrections are diluted

**Consequence**

OLAP value understates extremes

Risk of ecological fallacy


Aggregation masks within-cell heterogeneity, leading to biased group-level inference.

2. **Sparsity**

Sun–Lunch → NaN

Sat–Lunch → NaN

Sparse cells reduce statistical reliability and inflate variance in aggregated estimates.

3. Dimensionality Effects

If additional dimensions were added:

Number of cells explodes

More NaNs

Lower effective sample size per cell



**Result**

Curse of dimensionality

Trade-off between granularity and robustness



Increasing dimensionality improves segmentation but worsens sparsity and aggregation bias.

(e) Normalize residual-adjusted columns (size, tip, total bill) from (d). Compare Z-score, Min-Max, robust scaling; quantify impact on clustering stability and regression residual variance.

In [ ]:
tips_e = tips_encoded.copy()

# regression outputs from (a)
tips_e['residual'] = model.resid
tips_e['y_hat'] = model.fittedvalues

# residual-adjusted columns
tips_e['total_bill_adj'] = tips_e['y_hat'] + alpha * tips_e['residual']
tips_e['tip_adj'] = tips_e['tip'] + alpha * tips_e['residual']
tips_e['size_adj'] = tips_e['size'] + alpha * tips_e['residual']

In [ ]:
num_cols = ['total_bill_adj', 'tip_adj', 'size_adj']


from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

scalers = {
    'Z-score': StandardScaler(),
    'Min-Max': MinMaxScaler(),
    'Robust': RobustScaler()
}

scaled_sets = {}

for name, scaler in scalers.items():
  scaled_sets[name] = pd.DataFrame(
      scaler.fit_transform(tips_e[num_cols]),
      columns=num_cols,
      index=tips_e.index
  )

In [ ]:
#quantify cluster stability

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

cluster_stability = {}

for name, X_scaled in scaled_sets.items():
    kmeans = KMeans(n_clusters=3, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    cluster_stability[name] = silhouette_score(X_scaled, labels)

cluster_stability

In [ ]:
#quantify residual variance
import statsmodels.api as sm
import numpy as np

residual_variance = {}

for name, X_scaled in scaled_sets.items():
    X = sm.add_constant(X_scaled[['tip_adj', 'size_adj']])
    y = X_scaled['total_bill_adj']

    model_scaled = sm.OLS(y, X).fit()
    residual_variance[name] = np.var(model_scaled.resid)

residual_variance

(f) Perform 2-cluster k-means on normalized data from (e). Compute centroids, silhouette, Davies-Bouldin. Identify misassigned points and relate clusters to regression residuals.

In [ ]:
#prepare normalized data
X = scaled_sets['Robust'][['total_bill_adj', 'tip_adj', 'size_adj']]

In [ ]:
#Perform 2-Cluster K-Means
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=42)
cluster_labels = kmeans.fit_predict(X)
X['cluster'] = cluster_labels
centroids = kmeans.cluster_centers_
centroids

In [ ]:
#Compute Clustering Metrics
#(i) Silhouette Score
from sklearn.metrics import silhouette_score, davies_bouldin_score

sil_score = silhouette_score(X[['total_bill_adj', 'tip_adj', 'size_adj']], X['cluster'])
sil_score

In [ ]:
#(ii) Davies-Bouldin Index
db_score = davies_bouldin_score(X[['total_bill_adj', 'tip_adj', 'size_adj']], X['cluster'])
db_score

In [ ]:
#Identify Misassigned Points
import numpy as np

distances = kmeans.transform(X[['total_bill_adj', 'tip_adj', 'size_adj']])
# Distance to assigned centroid
assigned_distance = np.array([distances[i, label] for i, label in enumerate(X['cluster'])])
# Threshold: top 5% farthest points
misassigned_idx = X.index[assigned_distance > np.percentile(assigned_distance, 95)]
misassigned_points = X.loc[misassigned_idx]
misassigned_points

In [ ]:
import seaborn as sns
# tips_e has residuals from regression
X['residual'] = tips_e['residual']

sns.boxplot(x='cluster', y='residual', data=X)

(g) Using clusters from (f), cross-validate regression. Compare leave-one-out vs k-fold; quantify variance-bias tradeoff. Evaluate ensemble improvement with correlated predictors.

In [ ]:
import statsmodels.api as sm
X_reg = sm.add_constant(X_scaled[['tip_adj', 'size_adj']])

In [ ]:
X_scaled = scaled_sets['Robust']
y = X_scaled['total_bill_adj']

X_reg = sm.add_constant(X_scaled[['tip_adj', 'size_adj']])

In [ ]:
#Leave-One-Out Cross-Validation
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
import numpy as np

loo = LeaveOneOut()
mse_loo = []

lr = LinearRegression()

for train_idx, test_idx in loo.split(X_reg):
    lr.fit(X_reg.iloc[train_idx], y.iloc[train_idx])
    y_pred = lr.predict(X_reg.iloc[test_idx])
    mse_loo.append((y_pred - y.iloc[test_idx].values)**2)

mse_loo = np.mean(mse_loo)
print("LOO MSE:", mse_loo)

In [ ]:
#K-Fold Cross-Validation
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_kfold = []

for train_idx, test_idx in kf.split(X_reg):
    lr.fit(X_reg.iloc[train_idx], y.iloc[train_idx])
    y_pred = lr.predict(X_reg.iloc[test_idx])
    mse_kfold.append(mean_squared_error(y.iloc[test_idx], y_pred))

mse_kfold = np.mean(mse_kfold)
print("5-Fold CV MSE:", mse_kfold)

In [ ]:
loo_mse = 0.025859265273434757
kf_mse = 0.0263289515950672
percent_diff = (kf_mse - loo_mse) / loo_mse * 100
percent_diff

Using 5-Fold increases bias slightly (~1.8%) but reduces variance hence a more stable estimate

In [ ]:
X.columns

ensemble_preds = np.zeros(len(y))

for cluster in X['cluster'].unique():
  idx = X['cluster'] == cluster
  X_cluster = sm.add_constant(X_scaled.loc[idx, ['tip_adj','size_adj']])
  y_cluster = y[idx]

  lr.fit(X_cluster, y_cluster)
  ensemble_preds[idx] = lr.predict(X_cluster)

# Ensemble residual variance
ensemble_resid_var = np.var(y - ensemble_preds)
print(f"Ensemble residual variance: {ensemble_resid_var}")

# Global regression
X_global = sm.add_constant(X_scaled[['tip_adj','size_adj']])
model_global = sm.OLS(y, X_global).fit()
global_resid_var = np.var(model_global.resid)
print(f"Global variance: {global_resid_var}")

improvement = global_resid_var - ensemble_resid_var
print(f"Improvement: {improvement}")

(h) Design a star schema based on outputs from (g): fact table (total bill, tip, size), dimensions (sex, smoker, day, time). Include slowly changing dimensions.

In [ ]:
-- Fact Table
CREATE TABLE Fact_Tips (
    fact_id INT PRIMARY KEY,
    total_bill DECIMAL(10,2),
    tip DECIMAL(10,2),
    size INT,
    sex_id INT,
    smoker_id INT,
    day_id INT,
    time_id INT,
    cluster_id INT,
    FOREIGN KEY (sex_id) REFERENCES Sex_Dim(sex_id),
    FOREIGN KEY (smoker_id) REFERENCES Smoker_Dim(smoker_id),
    FOREIGN KEY (day_id) REFERENCES Day_Dim(day_id),
    FOREIGN KEY (time_id) REFERENCES Time_Dim(time_id),
    FOREIGN KEY (cluster_id) REFERENCES Cluster_Dim(cluster_id)
);

-- Dimension Tables
CREATE TABLE Sex_Dim (
    sex_id INT PRIMARY KEY,
    sex VARCHAR(10),
    effective_start_date DATE,
    effective_end_date DATE
);

CREATE TABLE Smoker_Dim (
    smoker_id INT PRIMARY KEY,
    smoker VARCHAR(5),
    effective_start_date DATE,
    effective_end_date DATE
);

CREATE TABLE Day_Dim (
    day_id INT PRIMARY KEY,
    day VARCHAR(10),
    effective_start_date DATE,
    effective_end_date DATE
);

CREATE TABLE Time_Dim (
    time_id INT PRIMARY KEY,
    time VARCHAR(10),
    effective_start_date DATE,
    effective_end_date DATE
);

CREATE TABLE Cluster_Dim (
    cluster_id INT PRIMARY KEY,
    cluster_name VARCHAR(20)
);

(i) Apply OLAP on the star schema from (h). Analyze revenue trends by day and time. Detect interactions and heterogeneous variance patterns.

In [ ]:
#create OLAP cube

fact_table = X.copy()  # X contains residual-adjusted measures + cluster + residual
fact_table = fact_table.merge(tips[['sex', 'smoker', 'day', 'time']], left_index=True, right_index=True)

# Create OLAP-style pivot table: total_bill by day and time
olap_cube = pd.pivot_table(
    fact_table,
    values='total_bill_adj',
    index='day',   # Day as row
    columns='time', # Time as column
    aggfunc='mean' # Can also use sum for revenue
)

olap_cube

In [ ]:
#analyze revenue trends by Day and Time
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
sns.heatmap(olap_cube, annot=True, fmt=".2f", cmap="YlGnBu")
plt.title("Revenue Trends by Day and Time (OLAP Cube)")
plt.show()

In [ ]:
#Detect Interactions (Day × Time)
from statsmodels.formula.api import ols

# Two-way ANOVA
formula = 'total_bill_adj ~ C(day) * C(time)'
model = ols(formula, data=fact_table).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
#Detect Heterogeneous Variance
# Group variance by day and time
variance_cube = fact_table.groupby(['day','time'])['total_bill_adj'].var().unstack()
variance_cube

In [ ]:
plt.figure(figsize=(8,5))
sns.heatmap(variance_cube, annot=True, fmt=".4f", cmap="Reds")
plt.title("Variance of Total Bill by Day and Time")
plt.show()

(j) Generate association rules from clustered and OLAP-adjusted data in (i) (e.g., size>3 ⇒ total bill>20). Compute lift, leverage, conviction; discuss overfitting risks.

In [ ]:
#prepare the dataset
# fact_table from (i)
data = fact_table.copy()

# Convert numeric columns to categorical for association rules
data['size_large'] = data['size_adj'] > 3
data['high_total_bill'] = data['total_bill_adj'] > 20
data['tip_high'] = data['tip_adj'] > 5  # optional threshold

# Include categorical dimensions
data['sex'] = data['sex']
data['smoker'] = data['smoker']
data['day'] = data['day']
data['time'] = data['time']
data['cluster'] = data['cluster']

# Select relevant columns for association rules
ar_data = data[['size_large','high_total_bill','tip_high','sex','smoker','day','time','cluster']]

# Convert all to strings for one-hot encoding
ar_data = ar_data.astype(str)

In [ ]:
#onehot encode
from mlxtend.preprocessing import TransactionEncoder

# Convert DataFrame to list of lists for TransactionEncoder
transactions = ar_data.apply(lambda x: list(x), axis=1).tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_hot = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
#generate frequent itemsets
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df_hot, min_support=0.1, use_colnames=True)
frequent_itemsets.sort_values('support', ascending=False).head(10)

In [ ]:
#generate association rules
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules = rules[['antecedents','consequents','support','confidence','lift','leverage','conviction']]
rules.sort_values('lift', ascending=False).head(10)

**Overfitting risks**

1.Small dataset

With few transactions, many rules may appear strong purely by chance

Confidence/lift can be inflated



2.Adding many categorical or derived columns implies combinatorial explosion which in turn implies spurious rules


3.Using residual-adjusted measures might exaggerate patterns, especially if residuals were extreme

**Mitigation**

Increase min_support (e.g., ≥10%)

Focus on rules with high lift and meaningful antecedents

Cross-validate rules on subsets or holdout data

(k) Apply the Minimum Description Length (MDL) principle using all prior analyses. Select among regression, clustering, and classification models. Quantify model complexity vs data-fit and justify selection.

In [ ]:
n = len(y)  # number of observations

#regression mdl
lr = LinearRegression()
lr.fit(X_scaled[['tip_adj','size_adj']], y)
resid = y - lr.predict(X_scaled[['tip_adj','size_adj']])
n_params = X_scaled[['tip_adj','size_adj']].shape[1] + 1  # +1 for intercept

# MDL formula (simplified, BIC-style)
mdl_regression = (n_params/2) * np.log(n) + (n/2) * np.log(np.var(resid))
print("Regression MDL:", mdl_regression)

#kmeans mdl
k = 2
features = X_scaled[['tip_adj','size_adj','total_bill_adj']].shape[1]
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X_scaled[['tip_adj','size_adj','total_bill_adj']])
inertia = kmeans.inertia_  # sum of squared distances

# MDL = model complexity + data fit
mdl_clusters = k * features * np.log(n) + (n/2) * np.log(inertia)
print("K-Means MDL:", mdl_clusters)

#Association rules mdl
# rules DataFrame should have 'antecedents' and 'confidence'
n_rules = len(rules)
avg_conditions = np.mean([len(a) for a in rules['antecedents']])

# Fit: log of confidence (smaller fit → better)
fit_ar = -np.sum(np.log(rules['confidence'] + 1e-9))  # add small number to avoid log(0)
mdl_rules = n_rules * avg_conditions + fit_ar
print("Association Rules MDL:", mdl_rules)

#comparison
mdl_comparison = pd.DataFrame({
    'Model': ['Regression', 'K-Means', 'Association Rules'],
    'MDL': [mdl_regression, mdl_clusters, mdl_rules]
}).sort_values('MDL')

print("\nMDL Comparison:\n", mdl_comparison)

# Selection: lowest MDL is preferred
best_model = mdl_comparison.iloc[0]['Model']
print("\nSelected Model by MDL:", best_model)

| Model             | Complexity | Fit (Residual/Error) | MDL Score |
| ----------------- | ---------- | -------------------- | --------- |
| Regression        | 3          | 0.025 (resid var)    | -440.99   |
| K-Means           | 6          | 690.90 (inertia)     | 690.90    |
| Association Rules | 2376       | 0.95 (high conf)     | 2376.64   |